In [ ]:
import json
import gc

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
from tqdm import tqdm

import lightgbm as lgbm

In [ ]:
SUBMISSION_RUN = True

In [ ]:
# Constants

# Paths
INPUT_BASE_PATH = "/kaggle/input/"
RAW_DATA_INPUT_PATH = f"{INPUT_BASE_PATH}/m5-forecasting-accuracy"
PROCESSED_DATA_INPUT_PATH = f"{INPUT_BASE_PATH}/m5-acc"
OUTPUT_BASE_BATH = "/kaggle/working"

# Timestamps
MAX_TRAIN_TIMESTAMP = 1941
START_TEST_TIMESTAMP = 1942
START_TEST_WM_YR_WK = 11617

# Data Input

In [ ]:
# Load raw data

# CALENDAR_DATA = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/calendar.csv")
# SELL_PRICES = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/sell_prices.csv")
# SALES_TRAIN_EVALUATION = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/sales_train_evaluation.csv")

SAMPLE_SUBMISSION = pd.read_csv(f"{RAW_DATA_INPUT_PATH}/sample_submission.csv")
SUBMISSION_INDEX = SAMPLE_SUBMISSION.set_index("id").index
VAL_SUBMISSION = SAMPLE_SUBMISSION[SAMPLE_SUBMISSION["id"].str.contains("validation")]
EVAL_SUBMISSION = SAMPLE_SUBMISSION[SAMPLE_SUBMISSION["id"].str.contains("evaluation")]

In [ ]:
# Load preprocessed data
train_data_set = pd.read_parquet(f"{PROCESSED_DATA_INPUT_PATH}/m5-acc-train.parquet")

In [ ]:
# Drop any columns that we dont need for training
cols_to_drop = ["av_store_dept_sales", "date", "wm_yr_wk"]
try:
    train_data_set = train_data_set.drop(columns=cols_to_drop)
except KeyError:
    print(f"Columns not found in axis. Skipping ...")

_ = gc.collect()

In [ ]:
# Cast to lower resolution types to save memory
def cast_data_types(dataset: pd.DataFrame) -> pd.DataFrame:
    # Category cols
    item_category_cols = ["dept_id", "cat_id", "store_id", "state_id"]
    date_category_cols = ["wm_yr_wk", "weekday", "month", "year", "event_name", "event_type", "snap_CA", "snap_TX", "snap_WI"]
    price_category_cols = ["item_on_sale"]
    for category_col in item_category_cols + date_category_cols + price_category_cols:
        try:
            dataset[category_col] = dataset[category_col].astype("category")
        except KeyError:
            print(f"Column {category_col} does not exist. Skipping ...")
    
    # Integer cols
    int_cols = ["d", "count"]
    for int_col in int_cols:
        try:
            dataset[int_col] = dataset[int_col].astype(np.int16)
        except KeyError:
            print(f"Column {int_col} does not exist. Skipping ...")
    
    # Float cols
    float_sales_cols = [
        "item_id",
        "count_lag_28", "count_lag_29", "count_lag_30", "count_lag_31",
        "av_store_dept_sales_lag_28", "av_store_dept_sales_lag_29", "av_store_dept_sales_lag_30", "av_store_dept_sales_lag_31"
    ]
    float_price_cols = [
        "sell_price",
        "sell_price_diff_1", "sell_price_diff_2", "sell_price_diff_3", "sell_price_diff_7",
        "av_item_state_sell_price", "av_dept_state_sell_price"
    ]
    float_date_cols = [
        "weekday_sin", "weekday_cos", "month_sin", "month_cos"
    ]
    float_cols = float_sales_cols + float_price_cols + float_date_cols
    for float_col in float_cols:
        try:
            dataset[float_col] = dataset[float_col].astype(np.float16)
        except KeyError:
            print(f"Column {int_col} does not exist. Skipping ...")
    
    return dataset

train_data_set = cast_data_types(train_data_set)

# LightGBM Model

In [ ]:
# Rolling window validation

def time_series_split(
    df: pd.DataFrame,
    n_folds: int = 5,
    horizon: int = 28,
    overlap: int = 0,
    max_timestamp: int = MAX_TRAIN_TIMESTAMP,
):
    min_timestamp = max_timestamp - n_folds * horizon + (n_folds - 1) * overlap
    for fold_idx in range(n_folds):
        start = min_timestamp + fold_idx * (horizon - overlap)
        stop = start + horizon

        train_data = df[df["d"] < start]
        valid_data = df[(df["d"] >= start) & (df["d"] < stop)]
        
        print(f"Fold index: {fold_idx}")
        print(f"Train idx (start, end): ({train_data['d'].min()}, {train_data['d'].max()})")
        print(f"Valid idx (start, end): ({valid_data['d'].min()}, {valid_data['d'].max()})")
        print("==================")
        
        yield train_data, valid_data
        
        
def train_predict_score(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    train_parameters: dict,
    dataset_parameters: dict | None = None,
) -> tuple[float, float]:
    dataset_parameters = dataset_parameters or {}
    train_data = lgbm.Dataset(data=X_train, label=y_train, **dataset_parameters)
    valid_data = lgbm.Dataset(data=X_test, label=y_test, **dataset_parameters)
    model = lgbm.train(
        train_parameters,
        num_boost_round=3000,
        train_set=train_data,
        valid_sets=[train_data, valid_data],
        callbacks=[
            lgbm.early_stopping(stopping_rounds=5),
            lgbm.log_evaluation(period=10),
        ]
    )
    
    y_hat = model.predict(X_test)
    y_hat = np.clip(y_hat, a_min=0, a_max=np.inf)
    rmse = np.sqrt(np.mean((y_test - y_hat) ** 2))

    return rmse, model.best_iteration

In [ ]:
CATEGORICAL_FEATURES = [
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
    "year",
    "event_name",
    "event_type",
    "snap_CA",
    "snap_TX",
    "snap_WI",
    "item_on_sale",
]
CONTINOUS_FEATURES = [
    "item_id",
    "weekday_sin",
    "weekday_cos",
    "month_sin",
    "month_cos",
    "sell_price",
    "sell_price_diff_1",
    "sell_price_diff_2",
    "sell_price_diff_3",
    "sell_price_diff_7",
#     "av_item_state_sell_price",
#     "av_dept_state_sell_price",
    "count_lag_28",
    "count_lag_29",
    "count_lag_30",
    "count_lag_31",
    "av_store_dept_sales_lag_28",
    "av_store_dept_sales_lag_29",
    "av_store_dept_sales_lag_30",
    "av_store_dept_sales_lag_31"
]
FEATURES = CATEGORICAL_FEATURES + CONTINOUS_FEATURES
LABEL = "count"

# Parameters
DATASET_PARAMETERS = {}
TRAIN_PARAMETERS = {
    "objective": "tweedie",
    'tweedie_variance_power': 1.1,
    "learning_rate": 0.05,
    "num_leaves": 2 ** 7 - 1,
    "max_bin": 2 ** 7 - 1,
    "bagging_freq": 1,
    "bagging_fraction": 0.8,
    "metric": "rmse",
    "force_col_wise": True,
}

In [ ]:
if not SUBMISSION_RUN:
    scores = []
    best_iterations = []
    
    for train, valid in tqdm(time_series_split(train_data_set, n_folds=5), total=5):
    
        X_train, y_train = train[FEATURES], train[LABEL]
        X_test, y_test = valid[FEATURES], valid[LABEL]
        score, best_iter = train_predict_score(
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            y_test=y_test,
            train_parameters=TRAIN_PARAMETERS,
            dataset_parameters=DATASET_PARAMETERS,
        )
        scores.append(score)
        best_iterations.append(best_iter)
    
    cv_results = {
        "scores": scores,
        "mean_score": np.mean(scores),
        "std_score": np.std(scores),
        "best_iterations": best_iterations,
        "mean_best_iteration": np.mean(best_iterations)
    }
    
    with open(f"{OUTPUT_BASE_BATH}/cv_results.json", "w") as f:
        json.dump(cv_results, f)
    
else:
    X_train, y_train = train_data_set[FEATURES], train_data_set[LABEL]
    train_data = lgbm.Dataset(data=X_train, label=y_train, **DATASET_PARAMETERS)
    model = lgbm.train(
        TRAIN_PARAMETERS,
        num_boost_round=450,
        train_set=train_data,
        valid_sets=[train_data],
        callbacks=[lgbm.log_evaluation(period=10)]
    )

# Forecast & Submit

In [ ]:
def pivot_forecast_df(X_test: pd.DataFrame, y_hat: np.ndarray) -> pd.DataFrame:
    forecast_df = pd.concat([X_test, pd.DataFrame(y_hat, columns=["count"])], axis=1)
    forecast_df = forecast_df.pivot(columns="d", index="id", values="count").sort_index(axis=1)
    forecast_df = forecast_df.rename_axis(None, axis=1).reset_index()
    return forecast_df

def rename_forecast_columns(forecast_df: pd.DataFrame, forecast_horizon: int = 28) -> pd.DataFrame:
    forecast_df = forecast_df.set_index("id")
    forecast_df.columns = [f"F{i}" for i in range(1, forecast_horizon + 1)]
    return forecast_df.reset_index()

In [ ]:
if SUBMISSION_RUN:
    test_data_set = pd.read_parquet(f"{PROCESSED_DATA_INPUT_PATH}/m5-acc-test.parquet")
    
    # Drop any columns that we dont need for training
    cols_to_drop = ["av_store_dept_sales", "date", "wm_yr_wk"]
    for col in cols_to_drop:
        try:
            test_data_set = test_data_set.drop([col], axis=1)
        except KeyError:
            print(f"Column '{col}' not found in axis. Skipping ...")
    _ = gc.collect()

    test_data_set = cast_data_types(test_data_set)

    y_hat = model.predict(test_data_set[FEATURES])
    y_hat = np.clip(y_hat, a_min=0, a_max=np.inf)

    forecast_df = pivot_forecast_df(test_data_set, y_hat)
    forecast_df = rename_forecast_columns(forecast_df)
    
    # Merge with sample submissions
    EVAL_SUBMISSION = EVAL_SUBMISSION[["id"]].merge(forecast_df, on="id", how="left")
    FINAL_SUBMISSIONS = pd.concat([VAL_SUBMISSION, EVAL_SUBMISSION])
    FINAL_SUBMISSIONS = FINAL_SUBMISSIONS.set_index("id").reindex(SUBMISSION_INDEX).reset_index()
    
    FINAL_SUBMISSIONS.to_csv(f"{OUTPUT_BASE_BATH}/submission.csv", index=False)